## Google Colab setup (don't run locally)

In [ ]:
import os
import shutil

# setting up paths
path_to_project_files = 'C:/Users/Jade/Downloads/Project-20250326T235242Z-001/Project'
existing = os.path.join(path_to_project_files, 'kaggle.json')
path_to_colab_utils = 'C:\Users\Jade\.kaggle'
target = os.path.join(path_to_colab_utils, 'kaggle.json')

# move the key to the colab root
os.makedirs(path_to_colab_utils, exist_ok=True)
shutil.copy(existing, target)
os.chmod(target, 600)

# download the data into /content (which is temporary)
!kaggle datasets download -d xhlulu/leafsnap-dataset -p path_to_project_files --unzip

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Jade\anaconda3\envs\ml_steam_proj\Scripts\kaggle.exe\__main__.py", line 7, in <module>
  File "C:\Users\Jade\anaconda3\envs\ml_steam_proj\Lib\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Jade\anaconda3\envs\ml_steam_proj\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 1734, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Jade\anaconda3\envs\ml_steam_proj\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [2]:
import sys

## Library Imports

In [3]:
from autoencoder import *
from dataloader import *
from cnn import *

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import os

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
import matplotlib.pyplot as plt

def showTensorInNotebook(tensor):
    """
    This takes a (3[RGB], H, W) tensor in R[0.0, 1.0] and displays it with matplotlib.
    """
    image = tensor.detach().cpu().numpy().transpose(1,2,0) # move the channel axis to the end, because PIL and matplotlib hate each other
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


## Building the data loader

These are transforms that allow us to ingest the image tensors with some extra confusion at training time. `processor` makes the data loader spit out tensors, and `noiser` adds Gaussian noise.

In [ ]:
# This just processes the images.
NOISE_RATIO = 0.1
H, W = 128, 128

processor = v2.Compose([
    v2.PILToTensor(), # the LeafsnapDataset class gives PIL Images, convert to torch Tensor
    v2.RandomRotation(degrees=(-90, 90), expand=True),
    v2.Resize((H, W)), # resize
    lambda x: x / 255.0, # convert N[0, 255] to R[0.0, 1.0]
    lambda x: torch.clip(x + NOISE_RATIO*torch.randn_like(x), 0.0, 1.0), # add noise
])

In [ ]:
BATCH_SIZE = 64

root_directory = os.path.join(os.getcwd(), 'leafsnap-dataset') # you make need to edit this path to work, though, it works on Colab by default and works locally if you keep the dataset at the root of the repo
image_paths_file = os.path.join(root_directory, "train.txt")
dataset = LeafsnapDataset(image_paths_file, root_directory, use_segmented=True, source="field", transform=processor)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

## Training the Convolutional Neural Network (CNN)

This model uses a modified version of ResNet from Homework 2. It specifically is a version of ResNet34, with kernel size raised to 5, and skip layers at sizes 32, 64, and 128. Images have been downscaled to 128x128, and the segmentation image is used as a 4th layer, resulting in an input size of 4x128x128.

In [ ]:
model = resnet(4, 185, device=device)

train_resnet_model(model, dataloader, 25, .001, device=device)


Epoch 1/20: 100%|██████████| 121/121 [05:10<00:00,  2.56s/it, batch=121/121, loss=6.58]


Epoch 0 loss: 10.936660147895498


Epoch 2/20: 100%|██████████| 121/121 [05:06<00:00,  2.53s/it, batch=121/121, loss=4.91]


Epoch 1 loss: 5.423137692380543


Epoch 3/20: 100%|██████████| 121/121 [05:05<00:00,  2.52s/it, batch=121/121, loss=4.71]


...

Epoch 17 loss: 1.6627394965857514


Epoch 19/20: 100%|██████████| 121/121 [05:09<00:00,  2.55s/it, batch=121/121, loss=1.48]


Epoch 18 loss: 1.5458257484041955


Epoch 20/20: 100%|██████████| 121/121 [05:08<00:00,  2.55s/it, batch=121/121, loss=1.69]

Epoch 19 loss: 1.4748163257748628


In [ ]:
torch.save(model.state_dict(), "cnn_model_train.pth")

## Testing the CNN

In [ ]:
correct_top1 = 0
correct_top5 = 0
total = 0

model.to(device)
with torch.no_grad(): # No gradients needed for evaluation
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        # Top-1 Accuracy
        _, predicted = torch.max(outputs, 1)
        correct_top1 += (predicted == labels).sum().item()

        # Top-5 Accuracy
        top5_preds = torch.topk(outputs, 5, dim=1).indices
        correct_top5 += torch.sum(top5_preds.eq(labels.view(-1, 1))).item()

        total += labels.size(0)

# Compute accuracies
top1_accuracy = 100 * correct_top1 / total
top5_accuracy = 100 * correct_top5 / total

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")


Top-1 Accuracy: 69.06%
Top-5 Accuracy: 89.99%


After various tweaks, I'm very happy with the current training accuracy of the CNN model, especially for the first check-in. Running at a 90% Top-5 accuracy is excellent, although there is certainly some more hyperparameter tweaking to be done. I may also test changing the model's structure, adding techniques such as dropout that have been used in other models for similar purposes. I would like to reach 80% Top-1 accuracy by the end.